In [ ]:
# In this lab, the objective is to identify the customers who were active in both May and June, and how did their activity differ between months. 
#To achieve this, follow these steps:
# Establish a connection between Python and the Sakila database.

import pandas as pd
from sqlalchemy import create_engine

def connect_to_sakila(username, password, host, port, database):
    """
    Establish a connection to the Sakila database.
    
    :param username: Database username
    :param password: Database password
    :param host: Database host
    :param port: Database port
    :param database: Database name
    :return: SQLAlchemy engine object
    """
    connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}'
    engine = create_engine(connection_string)
    return engine


# Write a Python function called rentals_month that retrieves rental data for a given month and year 
# (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

# engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
# month: an integer representing the month for which rental data is to be retrieved.
# year: an integer representing the year for which rental data is to be retrieved.
# The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

def rentals_month(engine, month, year):
    """
    Retrieve rental data for a given month and year from the Sakila database.
    
    :param engine: SQLAlchemy engine object
    :param month: Integer representing the month
    :param year: Integer representing the year
    :return: Pandas DataFrame containing the rental data
    """
    query = f"""
    SELECT rental_id, rental_date, inventory_id, customer_id, return_date, staff_id
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """
    with engine.connect() as connection:
        df = pd.read_sql(query, connection)
    return df


# Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input 
#along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

#The function should also include the month and year as parameters and use them to name the new column 
# according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".

#Hint: Consider making use of pandas groupby()

def rental_count_month(df, month, year):
    """
    Count the number of rentals made by each customer for a given month and year.
    
    :param df: DataFrame containing rental data
    :param month: Integer representing the month
    :param year: Integer representing the year
    :return: DataFrame with customer_id and the count of rentals
    """
    rental_count = df.groupby('customer_id').size().reset_index(name=f'rentals_{month:02}_{year}')
    return rental_count


# Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals 
#made by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, 
#which is the difference between the number of rentals in the two months.

def compare_rentals(df1, df2, month1, year1, month2, year2):
    """
    Compare the number of rentals made by each customer in two different months.
    
    :param df1: DataFrame containing rental counts for the first month
    :param df2: DataFrame containing rental counts for the second month
    :param month1: Integer representing the first month
    :param year1: Integer representing the first year
    :param month2: Integer representing the second month
    :param year2: Integer representing the second year
    :return: Combined DataFrame with rental counts and difference
    """
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer', suffixes=(f'_{month1:02}_{year1}', f'_{month2:02}_{year2}')).fillna(0)
    combined_df['difference'] = combined_df[f'rentals_{month1:02}_{year1}'] - combined_df[f'rentals_{month2:02}_{year2}']
    return combined_df